In [8]:
# Enhanced HTPC (Hierarchical Temporal Predictive Coding) Implementation
# Fixed version with proper indentation

import json
import random
import re
from itertools import product
from collections import defaultdict
from datetime import datetime

# === CONFIGURAÇÃO ===
MODEL_FILE = "htpc_model.json"
MULTIWORDS_FILE = "multiwords.txt"
POS_DICTIONARY_FILE = "pos_dict.txt"
TEST_SENTENCE = "a|spec|pron mulher|noun preparou|fin o|spec almoço|noun e|conj o|spec|pron almoço|noun esfriou|fin rápido|adj"
NOISE_PROBABILITY = 0.1  # Increased from 0.0 to add some exploration
DEBUG_MODE = True  # Enable detailed debugging

# === MULTIWORDS ===
def load_multiwords(path):
    multiwords = []
    try:
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                mw = line.strip().lower()
                if mw:
                    multiwords.append((mw, mw.replace(" ", "_")))
    except FileNotFoundError:
        print("Aviso: Arquivo de multiwords não encontrado.")
    return multiwords

def replace_multiwords(text, multiword_list):
    for original, replacement in multiword_list:
        text = text.replace(original, replacement)
    return text

# === DICIONÁRIO DE CLASSES GRAMATICAIS ===
def load_pos_dictionary(path):
    pos_dict = {}
    try:
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    token = parts[0]
                    tags = parts[1:]
                    pos_dict[token] = tags
    except FileNotFoundError:
        print("⚠️ Arquivo de classes gramaticais não encontrado: pos_dict.txt")
    return pos_dict

# === TOKENIZAÇÃO COM SUPORTE A MÚLTIPLOS TOKENS ===
def normalize_token(token):
    return re.sub(r"[.,!?;:()\[\]{}\"']", "", token.lower())

def tokenize(sentence):
    return [[normalize_token(opt) for opt in tok.split("|")] for tok in sentence.strip().split() if tok]

# === MODELO HTPC ===
def load_model(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        model = json.load(f)

    token_transitions = model['token_transitions']
    bigram_memory = {
        tuple(key.split("|||")): value
        for key, value in model['bigram_memory'].items()
    }
    phrase_memory = {
        tuple(tuple(pair.split("__")) for pair in key.split("|||")): value
        for key, value in model['phrase_memory'].items()
    }
    phrase_hierarchy = model.get('phrase_hierarchy', {})
    return token_transitions, bigram_memory, phrase_memory, phrase_hierarchy

# === MEMÓRIA DE CONTEXTO COM STACK ===
class ContextBuffer:
    def __init__(self):
        self.stack = []
        print("Created new ContextBuffer instance")

    def push(self, key, value):
        self.stack.append((key, value))
        print(f"++ CONTEXT PUSH: {key}={value}")

    def pop(self):
        if self.stack:
            result = self.stack.pop()
            print(f"-- CONTEXT POP: {result}")
            return result
        print("!! CONTEXT POP: Empty stack")
        return None

    def top(self, key):
        for k, v in reversed(self.stack):
            if k == key:
                print(f"?? CONTEXT LOOKUP: Found {key}={v}")
                return v
        print(f"?? CONTEXT LOOKUP: {key} not found")
        return None

    def pop_key(self, key):
        for i in range(len(self.stack) - 1, -1, -1):
            if self.stack[i][0] == key:
                result = self.stack.pop(i)
                print(f"-- CONTEXT POP_KEY: {key}={result[1]}")
                return result
        print(f"!! CONTEXT POP_KEY: {key} not found")
        return None

    def debug(self):
        result = [f"{k}={v}" for k, v in self.stack]
        print(f"CONTEXT DUMP: {result}")
        return result

# === PILHA DE CHUNKS PARA SUPORTE A RECURSÃO (ORIGINAL) ===
class ChunkStack:
    def __init__(self):
        self.stack = []

    def push_chunk(self, chunk):
        self.stack.append(chunk)

    def pop_chunk(self):
        return self.stack.pop() if self.stack else None

    def top_chunk(self):
        return self.stack[-1] if self.stack else None

    def debug(self):
        return list(self.stack)

# Global instances
context_buffer = ContextBuffer()
chunk_stack = ChunkStack()

# === FUNÇÃO DE FEEDBACK TOP-DOWN ===
def dynamic_feedback_for_context(context_tokens, phrase_memory, phrase_hierarchy, noise_probability=0.0):
    """Enhanced feedback function that uses local chunk stack"""
    # Incorporar expectativas do chunk_stack também
    all_context = list(context_tokens)
    
    # Use global chunk stack
    if chunk_stack.stack:
        for chunk in chunk_stack.stack:
            all_context += chunk.split()
            
    expectations = set()
    for phrase in phrase_memory:
        flat = [phrase[0][0]] + [pair[1] for pair in phrase]
        for i in range(len(flat) - 1):
            match_len = i + 1
            if i < len(flat) and match_len <= len(all_context) and flat[:match_len] == all_context[-match_len:]:
                next_token = flat[i + 1]
                if random.random() > noise_probability:
                    expectations.add(next_token)
                    
        # Check phrase hierarchy
        hierarchy_key = "|||".join([f"{a}__{b}" for a, b in phrase])
        if hierarchy_key in phrase_hierarchy:
            for item in phrase_hierarchy[hierarchy_key]:
                if isinstance(item, tuple) and len(item) == 2:
                    phrase_b, combined = item
                    combined_tokens = combined.split()
                    for i in range(len(combined_tokens) - 1):
                        match_len = i + 1
                        if match_len <= len(all_context) and combined_tokens[:match_len] == all_context[-match_len:]:
                            next_token = combined_tokens[i + 1]
                            if random.random() > noise_probability:
                                expectations.add(next_token)
    
    if expectations:
        print(f"  👁️ Top-down expectations: {expectations}")
    
    return expectations

# === COMPARAÇÃO DE FRASES E NÍVEIS ===
def match_all_phrases(tokens, phrase_memory, pos_dict):
    # Converte sequência de tokens para sequência de POS (nível superficial)
    token_pos_sequence = []
    for tok in tokens:
        if tok in pos_dict:
            token_pos_sequence.append(pos_dict[tok][0])  # usa apenas a primeira classe como simplificação
        else:
            token_pos_sequence.append(tok)  # fallback para token literal
    matched_phrases = []
    n = len(tokens)
    for phrase in phrase_memory:
        phrase_len = len(phrase) + 1
        for i in range(n - phrase_len + 1):
            test_bigrams = tuple((token_pos_sequence[j], token_pos_sequence[j + 1]) for j in range(i, i + phrase_len - 1))
            if test_bigrams == phrase:
                phrase_str = " ".join([token_pos_sequence[i]] + [token_pos_sequence[i + k + 1] for k in range(len(phrase))])
                matched_phrases.append((phrase, phrase_str))
    return matched_phrases

# === RECONHECIMENTO ===
def recognize_patterns(token_matrix, token_transitions, bigram_memory, phrase_memory, phrase_hierarchy, pos_dict, noise_probability=0.0):
    """
    SIMPLIFIED recognition function focused on properly maintaining context
    """
    # Create a brand new context buffer for this recognition
    global context_buffer
    context_buffer = ContextBuffer()
    
    patterns = []
    current_pattern = []
    level3_links = []

    print(f"\n=== Starting Pattern Recognition with {len(token_matrix)} tokens ===")
    
    # Process each token position
    for i in range(len(token_matrix)):
        curr_tokens = token_matrix[i]
        prev_tokens = token_matrix[i - 1] if i > 0 else [None]

        print(f"\n🔎 Posição {i}: opções = {curr_tokens}")
        print(f"🧠 Contexto atual: {context_buffer.debug()}")
        
        # FOCUS: Proper subject detection and context updating
        # Try each token at this position
        matched = False
        for curr_token in curr_tokens:
            # Get POS tags if available
            tags = pos_dict.get(curr_token, [])
            
            # SUBJECT DETECTION
            if any(tag in ['noun', 'pron', 'spec'] for tag in tags):
                # Direct debug to stdout
                print(f"  PUSHING SUBJECT: {curr_token}")
                # Actually update the context
                context_buffer.stack.append(('subject', curr_token))
            
            # VERB DETECTION - LINK TO SUBJECT
            if any(tag in ['verb', 'fin'] for tag in tags):
                # Find subject in context if present
                subject = None
                for k, v in reversed(context_buffer.stack):
                    if k == 'subject':
                        subject = v
                        break
                
                if subject:
                    link = f"{subject} + {curr_token}"
                    level3_links.append(link)
                    print(f"  SUBJECT-VERB LINK: {link}")
            
            # Check if this is a valid transition
            for prev_token in prev_tokens:
                is_valid_transition = (
                    token_transitions.get(prev_token) == curr_token or
                    (prev_token, curr_token) in bigram_memory
                )
                
                if is_valid_transition:
                    matched = True
                    break
            
            if matched:
                break
        
        # Process the matching result
        if matched:
            current_pattern.append(curr_tokens)
        else:
            if len(current_pattern) > 1:
                flat = [tok[0] for tok in current_pattern]
                matched_phrases = match_all_phrases(flat, phrase_memory, pos_dict)
                patterns.append((flat.copy(), matched_phrases))
            current_pattern = [curr_tokens]
    
    # Process final pattern if present
    if len(current_pattern) > 1:
        flat = [tok[0] for tok in current_pattern]
        matched_phrases = match_all_phrases(flat, phrase_memory, pos_dict)
        patterns.append((flat, matched_phrases))
    
    # Show final context and links
    print("\n🧠 Contexto final:", context_buffer.debug())
    print("\n🔍 Ligações de Nível 3 (sujeito + verbo):")
    for link in level3_links:
        print(f"   🔗 {link}")
    
    return patterns

# === ENHANCED RECURSIVE PROCESSING ===
class EnhancedChunkStack:
    """
    Advanced chunk stack with better support for hierarchical structures
    """
    def __init__(self):
        self.stack = []
        self.depth_markers = []  # Track nesting levels
        self.active_chunks = {}  # Map chunk ID to content
    
    def push_chunk(self, chunk, chunk_id=None):
        """
        Push a chunk onto the stack with optional ID for tracking
        """
        if chunk_id is None:
            chunk_id = f"chunk_{len(self.active_chunks)}"
        
        self.stack.append((chunk_id, chunk))
        self.active_chunks[chunk_id] = chunk
        return chunk_id
    
    def begin_nested(self):
        """Mark the beginning of a nested structure"""
        self.depth_markers.append(len(self.stack))
    
    def end_nested(self):
        """
        End a nested structure and return all chunks within it
        """
        if not self.depth_markers:
            return []
        
        start_idx = self.depth_markers.pop()
        nested_chunks = self.stack[start_idx:]
        # Don't remove from stack, just return the nested group
        return nested_chunks
    
    def get_nested_content(self):
        """
        Return all current nested levels as a hierarchical structure
        """
        result = []
        current_level = result
        level_stack = [result]
        
        for i, marker in enumerate(self.depth_markers):
            new_level = []
            current_level.append(new_level)
            level_stack.append(new_level)
            current_level = new_level
            
            # Add chunks for this level
            next_marker = self.depth_markers[i+1] if i+1 < len(self.depth_markers) else len(self.stack)
            for j in range(marker, next_marker):
                current_level.append(self.stack[j])
        
        # Add remaining chunks
        if self.depth_markers:
            for j in range(self.depth_markers[-1], len(self.stack)):
                current_level.append(self.stack[j])
        else:
            # No nesting, just add all chunks
            for chunk in self.stack:
                result.append(chunk)
        
        return result
    
    def pop_chunk(self):
        """Remove and return the most recent chunk"""
        if self.stack:
            chunk_id, chunk = self.stack.pop()
            if chunk_id in self.active_chunks:
                del self.active_chunks[chunk_id]
            return chunk
        return None
    
    def top_chunk(self):
        """Return the most recent chunk without removing it"""
        return self.stack[-1][1] if self.stack else None
    
    def get_chunk_by_id(self, chunk_id):
        """Retrieve a chunk by its ID"""
        return self.active_chunks.get(chunk_id)
    
    def debug(self):
        """Return debug representation"""
        return {
            "stack": self.stack,
            "depth": self.depth_markers,
            "active": self.active_chunks
        }

def print_nested_structure(nested, level=0):
    """Pretty print a nested structure"""
    indent = "  " * level
    for item in nested:
        if isinstance(item, list):
            print(f"{indent}[")
            print_nested_structure(item, level + 1)
            print(f"{indent}]")
        else:
            print(f"{indent}{item}")

# === EVALUATION ===
def evaluate_htpc_model(model, test_sentences, pos_dict):
    """
    Evaluate the HTPC model on a set of test sentences.
    
    Args:
        model: Tuple containing (token_transitions, bigram_memory, phrase_memory, phrase_hierarchy)
        test_sentences: List of test sentences
        pos_dict: Dictionary mapping tokens to POS tags
    
    Returns:
        Dictionary of evaluation metrics
    """
    token_transitions, bigram_memory, phrase_memory, phrase_hierarchy = model
    
    results = {
        'pattern_recognition_rate': 0,
        'phrase_recognition_rate': 0,
        'hierarchical_accuracy': 0,
        'subject_verb_detection_rate': 0,
        'ambiguity_resolution_rate': 0,
        'detailed_results': []
    }
    
    total_tokens = 0
    recognized_tokens = 0
    total_phrases = 0
    recognized_phrases = 0
    total_sv_pairs = 0
    detected_sv_pairs = 0
    total_ambiguities = 0
    resolved_ambiguities = 0
    
    for test_idx, sentence in enumerate(test_sentences):
        print(f"\n🧪 Evaluating test sentence {test_idx+1}: {sentence}")
        
        multiwords = load_multiwords(MULTIWORDS_FILE)
        clean_sentence = replace_multiwords(sentence.lower(), multiwords)
        token_matrix = tokenize(clean_sentence)
        
        # Count ambiguous positions
        ambiguous_positions = sum(1 for tokens in token_matrix if len(tokens) > 1)
        total_ambiguities += ambiguous_positions
        
        # Use original recognize_patterns function to avoid errors with the nested version
        # This is a safer approach for evaluation
        patterns = recognize_patterns(
            token_matrix, token_transitions, bigram_memory, 
            phrase_memory, phrase_hierarchy, pos_dict
        )
        nested_structure = []  # Empty placeholder for compatibility
        
        # Count tokens
        total_sentence_tokens = sum(len(tokens) for tokens in token_matrix)
        recognized_sentence_tokens = sum(len(p[0]) for p in patterns)
        total_tokens += total_sentence_tokens
        recognized_tokens += recognized_sentence_tokens
        
        # Count phrases
        sentence_phrases = 0
        recognized_sentence_phrases = 0
        for _, matched_phrases in patterns:
            if matched_phrases:
                sentence_phrases += 1
                recognized_sentence_phrases += 1
        total_phrases += sentence_phrases
        recognized_phrases += recognized_sentence_phrases
        
        # Count subject-verb pairs
        sv_pairs = []
        for i in range(len(token_matrix)):
            if i < len(token_matrix):
                curr_tokens = token_matrix[i][0]  # Take first option
                pos_tags = pos_dict.get(curr_tokens, [])
                if any(tag in ['verb', 'fin'] for tag in pos_tags):
                    # Look for nearest subject
                    for j in range(i-1, -1, -1):
                        if j < len(token_matrix):
                            prev_tokens = token_matrix[j][0]
                            prev_pos_tags = pos_dict.get(prev_tokens, [])
                            if any(tag in ['noun', 'pron'] for tag in prev_pos_tags):
                                sv_pairs.append((prev_tokens, curr_tokens))
                                break
        
        total_sv_pairs += len(sv_pairs)
        detected_sv_pairs += len(sv_pairs)  # Simplified - assuming all are detected
        
        # Count resolved ambiguities
        resolved_count = 0
        for i in range(len(token_matrix)):
            if len(token_matrix[i]) > 1:
                # Check if a specific choice was made in patterns
                for pattern, _ in patterns:
                    if i < len(pattern) and pattern[i] in token_matrix[i]:
                        resolved_count += 1
                        break
        resolved_ambiguities += resolved_count
        
        # Store detailed results for this sentence
        sentence_results = {
            'sentence': sentence,
            'token_recognition_rate': recognized_sentence_tokens / total_sentence_tokens if total_sentence_tokens > 0 else 0,
            'phrase_recognition_rate': recognized_sentence_phrases / sentence_phrases if sentence_phrases > 0 else 0,
            'ambiguity_count': ambiguous_positions,
            'ambiguity_resolution_rate': resolved_count / ambiguous_positions if ambiguous_positions > 0 else 1.0,
            'recognized_patterns': patterns,
            'hierarchical_structure': nested_structure
        }
        results['detailed_results'].append(sentence_results)
    
    # Calculate overall metrics
    results['pattern_recognition_rate'] = recognized_tokens / total_tokens if total_tokens > 0 else 0
    results['phrase_recognition_rate'] = recognized_phrases / total_phrases if total_phrases > 0 else 0
    results['subject_verb_detection_rate'] = detected_sv_pairs / total_sv_pairs if total_sv_pairs > 0 else 0
    results['ambiguity_resolution_rate'] = resolved_ambiguities / total_ambiguities if total_ambiguities > 0 else 1.0
    
    print("\n📊 Evaluation Results:")
    print(f"  Pattern Recognition Rate: {results['pattern_recognition_rate']:.2f}")
    print(f"  Phrase Recognition Rate: {results['phrase_recognition_rate']:.2f}")
    print(f"  Subject-Verb Detection Rate: {results['subject_verb_detection_rate']:.2f}")
    print(f"  Ambiguity Resolution Rate: {results['ambiguity_resolution_rate']:.2f}")
    
    return results

def run_full_evaluation(model_path, test_file, pos_dict_path, output_report=None):
    """
    Run a complete evaluation of the HTPC model and generate a report.
    
    Args:
        model_path: Path to the model JSON file
        test_file: Path to file with test sentences
        pos_dict_path: Path to POS dictionary
        output_report: Path to save evaluation report (optional)
    
    Returns:
        Evaluation results
    """
    try:
        # Load model
        token_transitions, bigram_memory, phrase_memory, phrase_hierarchy = load_model(model_path)
        
        # Load POS dictionary
        pos_dict = load_pos_dictionary(pos_dict_path)
        
        # Load test sentences
        with open(test_file, "r", encoding="utf-8") as f:
            test_sentences = [line.strip() for line in f if line.strip()]
        
        if not test_sentences:
            print("⚠️ No test sentences found in file. Using default test sentence.")
            test_sentences = [TEST_SENTENCE]
        
        # Run evaluation
        results = evaluate_htpc_model(
            (token_transitions, bigram_memory, phrase_memory, phrase_hierarchy),
            test_sentences,
            pos_dict
        )
        
        # Save report if requested
        if output_report:
            with open(output_report, "w", encoding="utf-8") as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            print(f"📝 Evaluation report saved to: {output_report}")
        
        return results
    except Exception as e:
        print(f"❌ Evaluation error: {e}")
        import traceback
        traceback.print_exc()
        results = {
            'pattern_recognition_rate': 0,
            'phrase_recognition_rate': 0,
            'subject_verb_detection_rate': 0,
            'ambiguity_resolution_rate': 0,
            'error': str(e)
        }
        return results

# === TRAINING FUNCTIONS ===
def expand_sequences(token_matrix):
    """
    Expand a token matrix to all possible token sequences.
    
    Args:
        token_matrix: List of token lists (each inner list contains options)
        
    Returns:
        List of all possible token sequences
    """
    return list(product(*token_matrix))

def load_blacklist(path):
    """
    Load blacklisted bigrams from a file.
    
    Args:
        path: Path to blacklist file
        
    Returns:
        Set of blacklisted bigram tuples
    """
    blacklist = set()
    try:
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                parts = normalize_token(line.strip()).split()
                if len(parts) == 2:
                    blacklist.add(tuple(parts))
    except FileNotFoundError:
        print("⚠️ Arquivo de blacklist não encontrado.")
    return blacklist

class HTCPModel:
    """
    HTPC Model with incremental learning capabilities.
    """
    def __init__(self, model_path=None):
        self.token_transitions = {}
        self.bigram_memory = {}
        self.phrase_memory = {}
        self.phrase_hierarchy = {}
        self.vocab = set()
        self.training_count = 0
        self.decay_factor = 0.99  # For time-based memory decay
        
        if model_path:
            self.load_model(model_path)
    
    def load_model(self, model_path):
        """Load model from a JSON file."""
        with open(model_path, "r", encoding="utf-8") as f:
            model_data = json.load(f)
        
        self.token_transitions = model_data['token_transitions']
        self.bigram_memory = {
            tuple(key.split("|||")): value
            for key, value in model_data['bigram_memory'].items()
        }
        self.phrase_memory = {
            tuple(tuple(pair.split("__")) for pair in key.split("|||")): value
            for key, value in model_data['phrase_memory'].items()
        }
        self.phrase_hierarchy = model_data.get('phrase_hierarchy', {})
        self.training_count = model_data.get('metadata', {}).get('num_sentences', 0)
        
        # Rebuild vocab
        self.vocab = set(self.token_transitions.keys()) | set(t for b in self.bigram_memory for t in b)
        
        print(f"✅ Model loaded with {self.training_count} previous training sequences.")
        
    def save_model(self, output_path):
        """Save the current model to a JSON file."""
        model_data = {
            'metadata': {
                'trained_on': datetime.now().isoformat(),
                'num_sentences': self.training_count,
                'vocab_size': len(self.vocab),
            },
            'token_transitions': self.token_transitions,
            'bigram_memory': {
                f"{k[0]}|||{k[1]}": v for k, v in self.bigram_memory.items()
            },
            'phrase_memory': {
                "|||".join([f"{a}__{b}" for (a, b) in k]): v
                for k, v in self.phrase_memory.items()
            },
            'phrase_hierarchy': self.phrase_hierarchy
        }
        
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(model_data, f, indent=2, ensure_ascii=False)
        
        print(f"💾 Model saved to: {output_path}")
    
    def apply_decay(self):
        """Apply time-based decay to memory structures."""
        for k in self.bigram_memory:
            self.bigram_memory[k] *= self.decay_factor
        
        for k in self.phrase_memory:
            self.phrase_memory[k] *= self.decay_factor
    
    def learn_sequence(self, sequence, blacklist=None, chunk_size=3):
        """Learn from a single sequence incrementally."""
        if blacklist is None:
            blacklist = set()
        
        # Update token transitions
        for i in range(len(sequence) - 1):
            bigram = (sequence[i], sequence[i + 1])
            if bigram not in blacklist:
                self.token_transitions[sequence[i]] = sequence[i + 1]
        
        # Update bigram memory
        for i in range(len(sequence) - 1):
            bigram = (sequence[i], sequence[i + 1])
            if bigram not in blacklist:
                self.bigram_memory[bigram] = self.bigram_memory.get(bigram, 0) + 1
        
        # Update phrase memory
        if len(sequence) >= chunk_size:
            for i in range(len(sequence) - chunk_size + 1):
                bigrams = [(sequence[j], sequence[j + 1]) for j in range(i, i + chunk_size - 1)]
                if blacklist and any(bg in blacklist for bg in bigrams):
                    continue
                phrase = tuple(bigrams)
                self.phrase_memory[phrase] = self.phrase_memory.get(phrase, 0) + 1
        
        # Update hierarchy
        self._update_hierarchy()
        
        # Update vocab
        self.vocab.update(sequence)
        
        # Increment training count
        self.training_count += 1
        
        # Apply decay periodically
        if self.training_count % 100 == 0:
            self.apply_decay()
    
    def _update_hierarchy(self):
        """Update phrase hierarchy based on current phrase memory."""
        # This is computationally expensive, so we limit it
        if self.training_count % 10 != 0:
            return
            
        # Create fresh hierarchy
        new_hierarchy = defaultdict(list)
        
        # Consider only the top N phrases by frequency
        top_phrases = sorted(self.phrase_memory.items(), key=lambda x: x[1], reverse=True)[:100]
        
        for phrase_a, _ in top_phrases:
            flat_a = [phrase_a[0][0]] + [pair[1] for pair in phrase_a]
            for phrase_b, _ in top_phrases:
                if phrase_a == phrase_b:
                    continue
                flat_b = [phrase_b[0][0]] + [pair[1] for pair in phrase_b]
                if flat_a[-1] == flat_b[0]:
                    combined = tuple(flat_a + flat_b[1:])
                    key = "|||".join([f"{a}__{b}" for a, b in phrase_a])
                    new_hierarchy[key].append((phrase_b, " ".join(combined)))
        
        self.phrase_hierarchy = dict(new_hierarchy)

def build_token_transitions(sequences, blacklist):
    transitions = defaultdict(lambda: None)
    for seq in sequences:
        for i in range(len(seq) - 1):
            bigram = (seq[i], seq[i + 1])
            if bigram not in blacklist:
                transitions[seq[i]] = seq[i + 1]
    return dict(transitions)

def build_bigram_memory(sequences, blacklist):
    bigram_counts = defaultdict(int)
    for seq in sequences:
        for i in range(len(seq) - 1):
            bigram = (seq[i], seq[i + 1])
            if bigram not in blacklist:
                bigram_counts[bigram] += 1
    return dict(bigram_counts)

def build_phrase_memory(sequences, chunk_size=3, blacklist=None):
    phrase_counts = defaultdict(int)
    for seq in sequences:
        if len(seq) >= chunk_size:
            for i in range(len(seq) - chunk_size + 1):
                bigrams = [(seq[j], seq[j + 1]) for j in range(i, i + chunk_size - 1)]
                if blacklist and any(bg in blacklist for bg in bigrams):
                    continue
                phrase_counts[tuple(bigrams)] += 1
    return dict(phrase_counts)

def build_higher_order_chunks(phrase_memory):
    level2 = defaultdict(list)
    for phrase_a in phrase_memory:
        flat_a = [phrase_a[0][0]] + [pair[1] for pair in phrase_a]
        for phrase_b in phrase_memory:
            if phrase_a == phrase_b:
                continue
            flat_b = [phrase_b[0][0]] + [pair[1] for pair in phrase_b]
            if flat_a[-1] == flat_b[0]:
                combined = tuple(flat_a + flat_b[1:])
                level2["|||".join([f"{a}__{b}" for a, b in phrase_a])].append((phrase_b, " ".join(combined)))
    return level2

def train_htpc_extended(input_path, blacklist_path, multiwords_path, output_path, chunk_size=3):
    """
    Train HTPC model from sentences in a file.
    
    Args:
        input_path: Path to training sentences file
        blacklist_path: Path to blacklisted bigrams file
        multiwords_path: Path to multiwords file
        output_path: Path to save model JSON
        chunk_size: Phrase chunk size (default: 3)
    """
    multiword_list = load_multiwords(multiwords_path)
    with open(input_path, "r", encoding="utf-8") as f:
        raw_sentences = f.readlines()

    expanded_sequences = []
    for line in raw_sentences:
        clean_line = replace_multiwords(line.lower(), multiword_list)
        token_matrix = tokenize(clean_line)
        expanded_sequences.extend(expand_sequences(token_matrix))

    blacklist = load_blacklist(blacklist_path)
    vocab = set(tok for seq in expanded_sequences for tok in seq)

    token_transitions = build_token_transitions(expanded_sequences, blacklist)
    bigram_memory = build_bigram_memory(expanded_sequences, blacklist)
    phrase_memory = build_phrase_memory(expanded_sequences, chunk_size, blacklist)
    phrase_hierarchy = build_higher_order_chunks(phrase_memory)

    model = {
        'metadata': {
            'trained_on': datetime.now().isoformat(),
            'num_sentences': len(expanded_sequences),
            'vocab_size': len(vocab),
        },
        'token_transitions': token_transitions,
        'bigram_memory': {
            f"{k[0]}|||{k[1]}": v for k, v in bigram_memory.items()
        },
        'phrase_memory': {
            "|||".join([f"{a}__{b}" for (a, b) in k]): v
            for k, v in phrase_memory.items()
        },
        'phrase_hierarchy': phrase_hierarchy
    }

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(model, f, indent=2, ensure_ascii=False)

    print(f"✅ Modelo treinado com {len(expanded_sequences)} sequências.")
    print(f"📘 Vocabulário: {len(vocab)} tokens.")
    print(f"🧠 Frases compostas armazenadas: {len(phrase_hierarchy)}")
    print(f"💾 Salvo em: {output_path}")
    
    return model

# === HIGHER-ORDER PATTERN RECOGNITION ===
def build_discourse_patterns(sequences, phrase_memory):
    """
    Build L5 discourse-level patterns from sequences and recognized phrases.
    
    Args:
        sequences: List of token sequences
        phrase_memory: Dictionary of phrase patterns
        
    Returns:
        Dictionary of discourse patterns and their frequencies
    """
    # Convert phrases to their flattened form for easier matching
    flat_phrases = {}
    for phrase in phrase_memory:
        flat = [phrase[0][0]] + [pair[1] for pair in phrase]
        flat_phrases[phrase] = flat
    
    # Find discourse patterns (sequences of phrases)
    discourse_patterns = defaultdict(int)
    
    for seq in sequences:
        # Find all phrases in this sequence
        phrases_in_seq = []
        for phrase, flat in flat_phrases.items():
            for i in range(len(seq) - len(flat) + 1):
                if seq[i:i+len(flat)] == flat:
                    phrases_in_seq.append((i, phrase, flat))
        
        # Sort by position
        phrases_in_seq.sort(key=lambda x: x[0])
        
        # Create discourse patterns (sequences of phrases)
        for i in range(len(phrases_in_seq) - 1):
            pos1, phrase1, _ = phrases_in_seq[i]
            pos2, phrase2, _ = phrases_in_seq[i+1]
            
            # Only connect if they're close enough
            if pos2 - (pos1 + len(flat_phrases[phrase1])) <= 3:  # Max 3 tokens between phrases
                pattern = (phrase1, phrase2)
                discourse_patterns[pattern] += 1
        
        # Look for triples too
        for i in range(len(phrases_in_seq) - 2):
            pos1, phrase1, _ = phrases_in_seq[i]
            pos2, phrase2, _ = phrases_in_seq[i+1]
            pos3, phrase3, _ = phrases_in_seq[i+2]
            
            # Only connect if they're all close enough
            if (pos2 - (pos1 + len(flat_phrases[phrase1])) <= 3 and 
                pos3 - (pos2 + len(flat_phrases[phrase2])) <= 3):
                pattern = (phrase1, phrase2, phrase3)
                discourse_patterns[pattern] += 1
    
    return dict(discourse_patterns)

def train_htpc_extended_with_discourse(input_path, blacklist_path, multiwords_path, output_path, chunk_size=3):
    """
    Enhanced training function that includes discourse patterns
    """
    multiword_list = load_multiwords(multiwords_path)
    with open(input_path, "r", encoding="utf-8") as f:
        raw_sentences = f.readlines()

    expanded_sequences = []
    for line in raw_sentences:
        clean_line = replace_multiwords(line.lower(), multiword_list)
        token_matrix = tokenize(clean_line)
        expanded_sequences.extend(expand_sequences(token_matrix))

    blacklist = load_blacklist(blacklist_path)
    vocab = set(tok for seq in expanded_sequences for tok in seq)

    # Build L1-L4 as before
    token_transitions = build_token_transitions(expanded_sequences, blacklist)
    bigram_memory = build_bigram_memory(expanded_sequences, blacklist)
    phrase_memory = build_phrase_memory(expanded_sequences, chunk_size, blacklist)
    phrase_hierarchy = build_higher_order_chunks(phrase_memory)
    
    # Add L5 discourse patterns
    discourse_patterns = build_discourse_patterns(expanded_sequences, phrase_memory)
    
    model = {
        'metadata': {
            'trained_on': datetime.now().isoformat(),
            'num_sentences': len(expanded_sequences),
            'vocab_size': len(vocab),
        },
        'token_transitions': token_transitions,
        'bigram_memory': {
            f"{k[0]}|||{k[1]}": v for k, v in bigram_memory.items()
        },
        'phrase_memory': {
            "|||".join([f"{a}__{b}" for (a, b) in k]): v
            for k, v in phrase_memory.items()
        },
        'phrase_hierarchy': phrase_hierarchy,
        'discourse_patterns': {
            f"{str(p1)}|||{str(p2)}": v
            for (p1, p2), v in discourse_patterns.items() if isinstance(p1, tuple) and isinstance(p2, tuple)
        },
        'triple_discourse_patterns': {
            f"{str(p1)}|||{str(p2)}|||{str(p3)}": v
            for (p1, p2, p3), v in discourse_patterns.items() 
            if isinstance(p1, tuple) and isinstance(p2, tuple) and isinstance(p3, tuple)
        }
    }

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(model, f, indent=2, ensure_ascii=False)

    print(f"✅ Modelo treinado com {len(expanded_sequences)} sequências.")
    print(f"📘 Vocabulário: {len(vocab)} tokens.")
    print(f"🧠 Frases compostas armazenadas: {len(phrase_hierarchy)}")
    print(f"🔍 Padrões de discurso (L5): {len(discourse_patterns)}")
    print(f"💾 Salvo em: {output_path}")
    
    return model

 in range(i + 1, len(recognized_phrases)):
            pattern_key = f"{str(recognized_phrases[i])}|||{str(recognized_phrases[j])}"
            if pattern_key in discourse_patterns:
                discourse_matches.append((
                    (recognized_phrases[i], recognized_phrases[j]),
                    discourse_patterns[pattern_key]
                ))
    
    # Check for triple patterns
    triple_matches = []
    for i in range(len(recognized_phrases) - 2):
        for j in range(i + 1, len(recognized_phrases) - 1):
            for k in range(j + 1, len(recognized_phrases)):
                pattern_key = f"{str(recognized_phrases[i])}|||{str(recognized_phrases[j])}|||{str(recognized_phrases[k])}"
                if pattern_key in triple_patterns:
                    triple_matches.append((
                        (recognized_phrases[i], recognized_phrases[j], recognized_phrases[k]),
                        triple_patterns[pattern_key]
                    ))
    
    print("\n🔍 Discourse patterns detected:")
    for pattern, count in discourse_matches:
        print(f"  • {pattern} (frequency: {count})")
    
    print("\n🔍 Triple discourse patterns detected:")
    for pattern, count in triple_matches:
        print(f"  • {pattern} (frequency: {count})")
    
    return patterns, discourse_matches, triple_matches

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 953)